# 2. Data Analysis & Preprocessing

This JupyterNotebook is part of an exercise series titled *Data Analysis & Preprocessing*.\
The series itself includes practical exercises for lectures *3. Getting to Know Your Data* and *4. Data Preprocessing*. 

This exercise series is divided into three parts. There will be one exercise session per part (= one part per week):

- **2.1.** [Getting to Know Your Data](./2.1-Getting-to-Know-Your-Data.ipynb) (*last weeks notebook*)
- **2.2.** Preprocessing - Data Cleaning & Data Integration (*this notebook*)
    - **2.2.1.** [Recognizing Dirty Data](#2.2.1.-Recognizing-Dirty-Data)
        - **2.2.1.1.** [Incomplete](#2.2.1.1.-Incomplete)
        - **2.2.1.2.** [Noisy](#2.2.1.2.-Noisy)
        - **2.2.1.3.** [Inconsistencies](#2.2.1.3.-Inconsistencies)
        - **2.2.1.4.** [Errors/Outlier](#2.2.1.4.-Errors/Outlier)
        - **2.2.1.5.** [Intentional](#2.2.1.5.-Intentional)
    - **2.2.2.** [Data Cleaning](#2.2.2.-Data-Cleaning)
    - **2.2.3.** [Data Integration](#2.2.3.-Data-Integration)
        - **2.2.3.1.** [Nominal Data](#2.2.3.1.-Nominal-Data)
        - **2.2.3.2.** [Numerical Data](#2.2.3.2.-Numerical-Data)
- **2.3.** [Preprocessing - Data Reduction, Data Transformation & Data Discretization](./2.3-Preprocessing-Data-Reduction-Transformation-and-Discretization.ipynb) (*notebook of the week after next*)

<div class="alert alert-block alert-warning">

**Important:**
    
Work on the respective part yourself **BEFORE** each exercise session. The exercise session is **NOT** intended to take a first look at the exercise sheet, but to solve problems students had while preparing the exercise sheet beforehand.
    
</div>

## 2.2. Preprocessing - Data Cleaning & Data Integration

In this part you will apply the theoretical knowledge gained in the first part of the lecture *4. Data Preprocessing*. 

In [ ]:
# Import the required libraries
import tempfile
import sqlite3
import os
import urllib.request
import scipy.stats
import pandas as pd

In [ ]:
# Create a temporary directory
dataset_folder = tempfile.mkdtemp()

# Build path to database
database_path = os.path.join(dataset_folder, "adventure-works.db")

# Get the database
urllib.request.urlretrieve(
    "https://github.com/FAU-CS6/KDD-Databases/raw/main/AdventureWorks/adventure-works.db",
    database_path,
)

# Open connection to the adventure-works.db
connection = sqlite3.connect(database_path)

In [ ]:
# Modify the database to contain dirty data
cursor = connection.cursor()
cursor.executescript(
    "UPDATE Person SET LastName = 'Doe';"
    "UPDATE Employee SET HireDate = STRFTIME('%Y-%m-%d %H:%M:%S', HireDate);"
    "UPDATE Employee SET BirthDate = STRFTIME('%Y-%m-%d %H:%M:%S', BirthDate);"
    "UPDATE Employee SET Gender = 'Male' WHERE Gender = 'M' AND NationalIDNumber LIKE \"%8\";"
    "UPDATE Employee SET Gender = 'Female' WHERE Gender = 'F' AND NationalIDNumber LIKE \"%7%\";"
    "UPDATE Employee SET BirthDate = STRFTIME('%Y-%m-%d', BirthDate) WHERE NationalIDNumber LIKE \"%2%\";"
    "UPDATE Employee SET SickLeaveHours = 2306 WHERE BusinessEntityID = 10;"
    "UPDATE Employee SET VacationHours = -12 WHERE BusinessEntityID = 21;"
    "UPDATE Person SET LastName = 'Doe';"
    "UPDATE Employee SET JobTitle = 'None' WHERE NationalIDNumber LIKE \"%1\";"
    "UPDATE Employee SET CurrentFlag = 0 WHERE NationalIDNumber = 658797903 OR NationalIDNumber = 974026903;"
    "UPDATE Employee SET HireDate = STRFTIME('%Y-%m-%d %H:%M:%S', DATE(BirthDate, '-10 year')) "
    'WHERE NationalIDNumber LIKE "2%";'
)

In [ ]:
# Create the dirty DataFrame(s)
# Employee DataFrame
dirty_employee_df = pd.read_sql_query(
    "SELECT NationalIDNumber, LoginID, OrganizationNode, JobTitle, BirthDate, MaritalStatus, Gender,"
    "HireDate, SalariedFlag, VacationHours, SickLeaveHours, CurrentFlag, PersonType, NameStyle, Title,"
    "FirstName, MiddleName, LastName, Suffix, EmailPromotion, AdditionalContactInfo, Demographics "
    "FROM Employee "
    "JOIN Person ON Employee.BusinessEntityID = Person.BusinessEntityID",
    connection,
    index_col="NationalIDNumber",
)

### 2.2.1. Recognizing Dirty Data

Recall that we did not look for dirty data in the `order_df`. Real world data rarely contains data without some 
inconsistencies or noise. For this reason, we will now look at the `dirty_employee_df` that contains obvious 
inconsistencies.

<div class="alert alert-block alert-info">

**Task 1:** 

Independently use the skills you learned in Part One to familiarize yourself with the `dirty_employee_df`. In doing so, try to identify as many problems as possible with the dataset at hand.
</div>

In [ ]:
# Use the methods you learned in Part One to familiarize yourself with dirty_employee_df
# (Create new code cells whenever necessary)

In [ ]:
# Since this task is very loosely defined, no 100% sample solution can be given here.

# But a minimum is to look at the shape of the DataFrame
dirty_employee_df.shape

In [ ]:
# And then at least take a quick look at the head of all attributes. (Part 1)
dirty_employee_df[dirty_employee_df.columns[0:10]].head(25)

In [ ]:
# And then at least take a quick look at the head of all attributes. (Part 2)
dirty_employee_df[dirty_employee_df.columns[10:21]].head(25)

#### 2.2.1.1. Incomplete

Incomplete data can take many different forms. If you look at the present data set, you will notice `None` values in various attributes. 

In [ ]:
# Print the columns containing at least one `None`
dirty_employee_df[
    [
        "OrganizationNode",
        "JobTitle",
        "Title",
        "MiddleName",
        "Suffix",
        "AdditionalContactInfo",
    ]
].head(25)

Not every `None` equates to missing data. Sometimes it is simply the correct information that the attribute value is "nothing". This can be seen in the six attributes presented. 

<div class="alert alert-block alert-info">

**Task 2:**

For each of the attributes at hand, consider whether the `None` values indicate incomplete information, or if `None` is justified.
</div>

Write down your solution here:

A 100% assessment of which attributes are correct and which are not usually requires a knowledge of a domain expert. Without an domain expert we can only make educated assumptions like the following:

<u>Most likely correct attributes:</u>

- <b>MiddleName:</b> Some `None` values in the MiddleName are quite normal as not every person might have a middle name. However, it is still possible that this value is missing.
- <b>Suffix:</b> Even if the first 25 tuples contain only `None` values, this column also contains values like `III` and `Jr.` Since also in the normal case not many people carry such a suffix in the name, it seems quite valid that there are also many `Null` values here.
- <b>AdditionalContactInfo:</b> Again, the `None` values definitely suggest that this is a valid value. It also sounds reasonable that "Additional" information does not always have to be given.

<u>Most likely incomplete attributes:</u>

- <b>JobTitle:</b> It seems strange that in a dataset of employees, some people do not have a JobTitle. This is most likely incomplete information.
- <b>Title:</b> If the non-`None` values in Title were academic titles such as "Dr." or "Prof." it would be expected that these titles do not actually exist for every employee. However, since titles such as "Mr." or "Ms." are also used, it can be assumed that information is simply missing. It should definitely be possible to specify these titles for each employee.

<u>Attributes difficult to assess:</u>

- <b>OrganizationNode:</b> There is also a `None` value in the OrganizationNode attribute. However, this is assigned to the CEO of all people (see JobTitle). It is quite possible that only employees with superiors should be assigned to an OrganizationNode. However, this could also be an error as the CEO may have been with the company the longest and may simply have forgotten to add his OrganizationNode.

#### 2.2.1.2. Noisy

Noisy data, i.e. small measurement inaccuracies, are difficult to detect in the context of such an exercise. However, it is almost certain that the present data set does not contain any noisy data. 

<div class="alert alert-block alert-info">

**Task 3:**
    
Consider why noisy data is unlikely to be included in `dirty_employee_df`.
</div>

Write down your solution here:

The `dirty_employee_df` does not contain measured data. It is therefore extremely unlikely that noise is included in the data.

#### 2.2.1.3. Inconsistencies

Examples of inconsistencies of this data set can be found in the attributes `Gender`, `BirthDate`, and `HireDate`. 

<div class="alert alert-block alert-info">

**Task 4:**
    
Print the head of the attributes `Gender`, `BirthDate`, and `HireDate`.
</div>

In [ ]:
# Print the head of "Gender", "BirthDate", and "HireDate"

In [ ]:
# Print the head of "Gender", "BirthDate", and "HireDate"
dirty_employee_df[["Gender", "BirthDate", "HireDate"]].head(25)

<div class="alert alert-block alert-info">

**Task 5:**
    
Consider what inconsistencies are in the `Gender` attribute.
</div>

Write down your solution here:

The Gender in the data set is partly given as `F` and `M` and partly as `Female` and `Male`.

<div class="alert alert-block alert-info">

**Task 6:**
    
Consider what inconsistencies are in the `BirthDate` attribute.</div>

Write down your solution here:

The format in which the `BirthDate` has been specified differs.

<div class="alert alert-block alert-info">

**Task 7:**
    
Consider what inconsistencies are in the `HireDate` attribute. (Hint: Consider the attribute in conjunction with the other two attributes)</div>

Write down your solution here:

The `HireDate` is partly before the birthday. This is first of all an inconsistency, but the probability is high that this is actually even an error.

#### 2.2.1.4. Errors/Outlier

Errors in numerical values and in the special outlier are sometimes not quite detectable at a glance. Fortunately, we have already learned methods in Part One that we can now use.

Let's first consider the `SickLeaveHours` attribute. First, let's look at the `SickLeaveHours` attribute. Does it contain outliers or errors?

<div class="alert alert-block alert-info">

**Task 8:**
    
Use a boxplot diagram to graphically analyze whether `SickLeaveHours` contains Outliers.
</div>

In [ ]:
# Draw a boxplot diagram for "SickLeaveHours"

In [ ]:
# Draw a boxplot diagram for "SickLeaveHours"
dirty_employee_df["SickLeaveHours"].plot.box()

<div class="alert alert-block alert-info">

**Task 9:**
    
Think about a way to find out which tuple contains the outlier in `SickLeaveHours`.
</div>

In [ ]:
# Output the tuple containing the outlier

In [ ]:
# It is clear from the boxplot diagram that the SickLeaveHours of the Outlier are above 2000. This can be used:
dirty_employee_df[dirty_employee_df["SickLeaveHours"] > 2000]

Additionally, let's take a look at `VacationHours`.

<div class="alert alert-block alert-info">

**Task 10:**
    
Use a boxplot diagram to graphically analyze whether `VacationHours` contains Outliers.
</div>

In [ ]:
# Draw a boxplot diagram for "VacationHours"

In [ ]:
# Draw a boxplot diagram for "VacationHours"
dirty_employee_df["VacationHours"].plot.box()

<div class="alert alert-block alert-info">

**Task 11:**
    
Even though the boxplot diagram does not show any outliers, it clearly indicates a possible error in `VacationHours`. Which error?
</div>

Write down your solution here:

The minimum whisker extends to below 0 . It seems strange that hours can also become negative.

<div class="alert alert-block alert-info">

**Task 12:**
    
Output the affected tuples.
</div>

In [ ]:
# Output the tuple(s) containing the error

In [ ]:
# Output the tuple(s) containing the error
dirty_employee_df[dirty_employee_df["VacationHours"] < 0]

#### 2.2.1.5. Intentional

There is also an intentional error in the `dirty_employee_df`. It can be found in either the `MaritalStatus`, `SalariedFlag`, `FirstName`, or `LastName` attribute.

<div class="alert alert-block alert-info">

**Task 13:** 
    
Again, independently use your learned skills to search the attributes `MaritalStatus`, `SalariedFlag`, `FirstName`, and `LastName` for the intentional error.
</div>

In [ ]:
# Search for the intentional error

In [ ]:
# Search for the intentional error
dirty_employee_df[["MaritalStatus", "SalariedFlag", "FirstName", "LastName"]].head(25)

<div class="alert alert-block alert-info">

**Task 14:**
    
In which of the attributes is the intentional error to be found.
</div>

<b>The attribute with the intentional error:</b>
1. [ ] MaritalStatus
2. [ ] SalariedFlag
3. [ ] FirstName
4. [ ] LastName

<b>The attribute with the intentional error:</b>
1. [ ] MaritalStatus
2. [ ] SalariedFlag
3. [ ] FirstName
4. [X] LastName

<div class="alert alert-block alert-info">

**Task 15:**
    
Why do you think is this error intentional?
</div>

Write down your solution here:

For data protection reasons, it may sometimes be necessary to anonymize data. While it is rather atypical that this is the case with an employee database, if the data set had been issued for external analysis, for example, and it contained salary data, then one could explain such anonymization. However, it would be quite advantageous if not only this one attribute had been anonymized then.

### 2.2.2. Data Cleaning

The mere detection of dirty data is, of course, only the first step in the data cleaning process. While it is a best case scenario to correct dirty data step by step once it has been identified, this is often a lengthy and difficult process. 

In our example from section [Incomplete](#Incomplete), for example, only inconsistencies in `Gender` and `BirthDate` can be quickly fixed.

<div class="alert alert-block alert-info">

**Task 16:**
    
Replace all occurrences of `Female` with `F` and all occurrences of `Male` with `M` in the `Gender` attribute of the `dirty_employee_df`. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html))
</div>

In [ ]:
# Replace "Female" and "Male" values in "Gender"

In [ ]:
# Replace "Female" and "Male" values in "Gender"
dirty_employee_df.replace({"Gender": {"Female": "F", "Male": "M"}}, inplace=True)

<div class="alert alert-block alert-info">

**Task 17:**
    
Delete the suffix `00:00:00` in the `BirthDate` attribute. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) - Hint: You may want to use a regular expression.)
</div>

In [ ]:
# Delete the suffix 00:00:00 in BirthDate

In [ ]:
# Delete the suffix 00:00:00 in BirthDate
dirty_employee_df.replace(
    {"BirthDate": r"\ 00:00:00"}, {"BirthDate": ""}, regex=True, inplace=True
)

If only individual tuples contain an error/outlier and these cannot be manually fixed, the most efficient approach is often to simply remove these tuples from the dataset. 

For example this would apply to the tuples with `NationalIDNumber` 243322160 and 879342154 in the `dirty_employee_df`.

<div class="alert alert-block alert-info">

**Task 18:**
    
Delete the tuples with `NationalIDNumber` 243322160 and 879342154. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html))
</div>

In [ ]:
# Delete the tuples with NationalIDNumber 243322160 and 879342154

In [ ]:
# Delete the tuples with NationalIDNumber 243322160 and 879342154
dirty_employee_df.drop("243322160", inplace=True)
dirty_employee_df.drop("879342154", inplace=True)

Even for attributes that do not contain any information, it usually makes more sense to remove them from the data set. 

In this example this is the case with the attribute `LastName`.

<div class="alert alert-block alert-info">

**Task 19:**
    
Delete the attribute `LastName`. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html))
</div>

In [ ]:
# Delete the attribute LastName

In [ ]:
# Delete the attribute LastName
dirty_employee_df.drop(columns=["LastName"], inplace=True)

### 2.2.3. Data Integration

In the context of data integration, we mainly looked at correlation in the lecture. Calculation of this depends on the type of data.

#### 2.2.3.1. Nominal Data

One of the two data types we looked at in more detail in the lecture is nominal data. This describes all data that that is used to label variables without providing any quantitative value.

The first combination of nominal attributes we will look at in this section are `Gender` and `CurrentFlag`. We start by displaying the contigency table for these attributes.

<div class="alert alert-block alert-info">

**Task 20:**
    
Use the pandas function `crosstab()` to create a contingency table for the attributes `Gender` and `CurrentFlag`. Show the subtotals one time and once not. (Help: [pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html))
</div>

In [ ]:
# Display a contingency table without subtotals

In [ ]:
# Display a contingency table with subtotals

In [ ]:
# Display a contingency table without subtotals
pd.crosstab(dirty_employee_df["Gender"], dirty_employee_df["CurrentFlag"])

In [ ]:
# Display a contingency table with subtotals
pd.crosstab(
    dirty_employee_df["Gender"],
    dirty_employee_df["CurrentFlag"],
    margins=True,
)

The disadvantage of this contingency table is, of course, that only the observed quantities are displayed. For the calculation of the correlation, however, the expected quantities are also important. These can be calculated for example with the function `expected_freq()` from SciPy.

<div class="alert alert-block alert-info">

**Task 21:**
    
Use `expected_freq()` to output the expected quantities for the attributes `Gender` and `CurrentFlag`. (Help: [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.contingency.expected_freq.html#scipy.stats.contingency.expected_freq))
</div>

In [ ]:
# Display the expected quantities

In [ ]:
# Display the expected quantities
# Sample solution 1: Use pd.crosstab
pd_crosstab = pd.crosstab(dirty_employee_df["Gender"], dirty_employee_df["CurrentFlag"])

# Compute the expected_frequencies
# (it is fine if you just output them - the creation of a pd.DataFrame is just a bonus)
pd_expected_frequencies = scipy.stats.contingency.expected_freq(pd_crosstab)

# Create a pd.DataFrame
pd.DataFrame(
    data=pd_expected_frequencies, index=pd_crosstab.index, columns=pd_crosstab.columns
)

In [ ]:
# Sample solution 2: Use scipy.stats.contingency.crosstab
sp_crosstab_elements, sp_crosstab_count = scipy.stats.contingency.crosstab(
    dirty_employee_df["Gender"], dirty_employee_df["CurrentFlag"]
)

# Compute the expected_frequencies
# (it is fine if you just output them - the creation of a pd.DataFrame is just a bonus)
sp_expected_frequencies = scipy.stats.contingency.expected_freq(sp_crosstab_count)

# Create a pd.DataFrame
pd.DataFrame(
    data=sp_expected_frequencies,
    index=sp_crosstab_elements[0],
    columns=sp_crosstab_elements[1],
)

One can see that both the expected and observed values are quite close indicating a small correlation. 

However, in the lecture, the Chi-squared test was presented as a method to validate this more accurately.

<div class="alert alert-block alert-info">

**Task 22:**
    
Use the function `chi2_contingency` from SciPy to determine the correlation between `Gender` and `CurrentFlag`. (Help: [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html#scipy.stats.chi2_contingency))
</div>

In [ ]:
# Compute chi-squared for "Gender" and "CurrentFlag"

In [ ]:
# Compute chi-squared for "Gender" and "CurrentFlag"
scipy.stats.chi2_contingency(pd_crosstab)

<div class="alert alert-block alert-info">

**Task 23:**
    
Find out what the different values in the above output of `chi2_contingency` stand for and describe how to interpret them in this case.
</div>

Write down your solution here:

- <b>First value (0.0):</b><br />
This value represents the actual Chi-squared value. The closer it is to 0, the less correlation there is between attributes. Since this value is 0.0, we can assume there is no correlation between "Gender" and "CurrentFlag". (However: see explanation of the second value)

- <b>Second value (1.0):</b><br />
This is the so-called p-value. If this value is higher than the selected level of statistical significance (usually 0.01, 0.05 or 0.10), the chi-squared value is not fully reliable. Since in this case the value is significantly higher than any normally selected value, it can be assumed that our chi-squared should be viewed with great caution. 

- <b>Third value (1):</b><br />
The third value describes the degrees of freedom. This value is related to the number of categories (number of categories minus one). So there are two categories in our case (of course we already knew this from the contingency table).

- <b>Last value (Array):</b><br />
This is just another version of the expected values known from the previous task.

Second, let's look at the `Gender` and `SalariedFlag` attributes.

<div class="alert alert-block alert-info">

**Task 24:**
    
Using the methods learned above, calculate Chi-squared for `Gender` and `SalariedFlag`.
</div>

In [ ]:
# Compute chi-squared for "Gender" and "SalariedFlag"

In [ ]:
# Compute chi-squared for "Gender" and "SalariedFlag"
pd_crosstab_salaried = pd.crosstab(
    dirty_employee_df["Gender"], dirty_employee_df["SalariedFlag"]
)
scipy.stats.chi2_contingency(pd_crosstab_salaried)

<div class="alert alert-block alert-info">

**Task 25:**
    
Interpret the chi-squared for `Gender` and `SalariedFlag`.
</div>

Write down your solution here:

The two most important values for interpretation are the actual chi-squared value and the p-value. 

The p-value in this case is just above 0.05, so it does not yet reach this level of statistical significance. Depending on how certain one wants to be in drawing conclusions, one can either accept or reject the result of the chi-squared test.

The chi-squared value of about 3.65 is clearly above 0, so it is much more correlation than in the combination `Gender` and `CurrentFlag`.

#### 2.2.3.2. Numerical Data

The other data type we have considered in the context of correlation is the numeric data type. Here, it is suitable to look at the connection between `VacationHours` and `SickLeaveHours`. 

A method of graphical analysis of correlation in numerical data should already be known from Part One.

<div class="alert alert-block alert-info">

**Task 26:**
    
Draw a scatter plot regarding `VacationHours` and `SickLeaveHours`.
</div>

In [ ]:
# Draw a scatter plot regarding "VacationHours" and "SickLeaveHours"

In [ ]:
# Draw a scatter plot regarding "VacationHours" and "SickLeaveHours"
dirty_employee_df.plot.scatter(x="VacationHours", y="SickLeaveHours")

However, what was not part of the method in Part One is the interpretation of this diagram.

<div class="alert alert-block alert-info">

**Task 27:**
    
Interpret the scatter plot regarding `VacationHours` and `SickLeaveHours`.
</div>

Write down your solution here:

The scatter plot shows positive correlation between both variables indicating that employees who have been on more vacation are sick more often. 

**Caution in interpretation:** This may also be due to the fact that both `VacationHours` and `SickLeaveHours` were simply added over the contract period. We did not consider this in this analysis.

We can, of course, evaluate this graphical analysis mathematically. In the lecture we used Pearson's product-moment coefficient for this purpose.

<div class="alert alert-block alert-info">

**Task 28:**
    
Compute pearson's product-moment coefficient for `VacationHours` and `SickLeaveHours`. Use SciPy's `pearsonr()` function. (Help: [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html))
</div>

In [ ]:
# Compute pearson's product-moment coefficient for "VacationHours" and "SickLeaveHours"

In [ ]:
# Compute pearson's product-moment coefficient for "VacationHours" and "SickLeaveHours"
scipy.stats.pearsonr(
    dirty_employee_df["VacationHours"],
    dirty_employee_df["SickLeaveHours"],
)

<div class="alert alert-block alert-info">

**Task 29:**
    
Find out what the different values in the above output of `pearsonr` stand for and describe how to interpret them in this case.
</div>

Write down your solution here:

- <b>First value (approx. 0.989):</b><br />
This value represents the actual Pearson’s correlation coefficient. If it is positive, then there is a positive correlation between the two attributes. Our graphical analysis is confirmed here.

- <b>Second value (approx. 0.000):</b><br />
Similar to the chi-squared test, this is the p-value. Since this is virtually zero in this case, a high statistical significance can be assumed.